 ## COMP41680 Assignment 2: Text Classification  
 
 Submitted By : Farooq Shaikh
 
 Student Number: 19200161

#### Description:
The assignment involves the scraping of the consumer reviews from a webpage anf trainig a model to classify the reviews as either Positive or Negative. 
The aasignment has been divided in following three sections:

<font color='red'>Section 1:</font> Scraping of the reviews for three categories of the business, namely Automobile, Fashion and Cafe. Once Extracted we Text preprocessing on it and store the clean reiview text in a CSV for future use.

<font color='red'>Section 2:</font> This section involves building and testing of classification model on each of the category of the review. We build three different models employing three classification algorithm; Random forest, Naive Bayes, Logstic Regression. Each of the model is tested on each category to find the best performing model for each category.

<font color='red'>Section 3:</font> The final section involves cross category testing of the models. We use the trained model from one category say, Automobile and then test it on the other two categories to see if the models are able to genralize across the categories. 


In [ ]:
# importing the required libraries

import requests
import bs4
import urllib.request
import pandas as pd
from nltk.tokenize.treebank import TreebankWordDetokenizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from sklearn.metrics import f1_score
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
import re
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from time import time
from IPython.display import HTML
import pickle

In [ ]:
# specify the address of the webpage 
web_url= 'http://mlg.ucd.ie/modules/yalp'

In [15]:
# function to get the webpage html
def get_html_parsed(url):
    result  = requests.get(url)
    parsed_result = bs4.BeautifulSoup(result.text)
    return parsed_result

In [16]:
# display the page to see the tags present for scraping 
page1=get_html_parsed(web_url)
page1

<!DOCTYPE html>
<html lang="en">
<head>
<meta content="noindex" name="robots"/>
<meta content="Content on this site is posted for teaching purposes only. Original data is from yelp.com" name="description"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Yalp Home</title>
<link href="images/favicon.ico" rel="shortcut icon"/>
<!-- Bootstrap core CSS -->
<link href="assets/css/bootstrap.css" rel="stylesheet"/>
<!-- Custom styles for this template -->
<link href="assets/css/style.css" rel="stylesheet"/>
<link href="assets/css/font-awesome.min.css" rel="stylesheet"/>
<script src="assets/js/modernizr.js"></script>
</head>
<body>
<div class="container mtb">
<div class="row">
<div class="col-md-12">
<h3 class="info"><a class="info" href="index.html">Yalp</a> — Home</h3>
</div>
</div>
<div class="row">
<div class="col-md-10" id="content">
<div id="all">
<p>Select a business category from

In [17]:
# A function to extract all the links present in the webpage based on the tag
def links_extracter(param):
    all_links = []
    soup_parser = bs4.BeautifulSoup(str(param),"html.parser")
    for url in soup_parser.findAll('a'):
        all_links.append(url.get('href'))
    return all_links

In [18]:
# display all the extracted links in the form of a dataframe
links=links_extracter(page1)
links.pop(0)
links_display=links
links_display = [x[:-10] for x in links_display]
links_df=pd.DataFrame(links_display)
links_df.index.names = ['Link Number']
links_df.columns = ['Business Category']

In [19]:
links_df

,Business Category
Link Number,
0,automotive
1,cafes
2,fashion
3,gym
4,hair_salons
5,hotels
6,restaurants


In [20]:
# Combine all the business from one category 
def get_business_home_page_urls(link_number):
    category_page_part_url = web_url+'/'+links[link_number]
    # get links of each individual buisness page url
    business_pages_url= links_extracter(get_html_parsed(category_page_part_url))
    # combine the link with base URL to get link of each automobile business home page
    all_business_pages_urls=[]
    for link in business_pages_url :
        all_business_pages_urls.append(web_url+'/'+link)
    all_business_pages_urls.pop(0) # remove the first index.html one since it is useless
    all_business_pages_urls[:5] 
    return all_business_pages_urls

In [21]:
# We select the link number from the above DF
automobile_pages_urls = get_business_home_page_urls(0)
cafes_pages_urls=get_business_home_page_urls(1)
fashion_pages_urls=get_business_home_page_urls(2)

In [22]:
automobile_pages_urls[0:5]

['http://mlg.ucd.ie/modules/yalp/review_set_BS4u8wkRb70FDBZcYkJ9kQ.html',
 'http://mlg.ucd.ie/modules/yalp/review_set_3O2HLFpUZKWxE55Mpzg81g.html',
 'http://mlg.ucd.ie/modules/yalp/review_set_jeG-c5GwD0Uf0Ou5DLFTZg.html',
 'http://mlg.ucd.ie/modules/yalp/review_set_UGQEXFibRZIebMRkuJqh3Q.html',
 'http://mlg.ucd.ie/modules/yalp/review_set_o1IS7-SvFXmijKDc6UY55Q.html']

In [23]:
def review_rating_extract(category_url):    
    business_reviews_df=[] 
    reviews=[] 
    ratings=[] 
    for each_link in category_url:
        html = get_html_parsed(each_link)
        parser = bs4.BeautifulSoup(str(html),"html.parser")
        # Text from the review is extracted based on the tag present 
        for each_review in parser.findAll('p',class_='review-text'):
            reviews.append(each_review.text)
        # the rating is found from the img and alt tag 
        for each_rating in parser.findAll('img'):
            if each_rating.get('alt')!=None:
                # if the stars are less than 4 (0,1,3) then we term it as negative else positive
                if int(each_rating.get('alt')[0])<4:
                    ratings.append('negative')
                else:
                    ratings.append('positive')  
        business_reviews_df=(pd.DataFrame(reviews)) 
        business_reviews_df['Rating']=ratings  
        business_reviews_df.columns=['Review','Rating']
    return business_reviews_df

In [ ]:
# extract all the reviews from the category URL by passing it to the review_rating _extract function
automobile_reviews_df= review_rating_extract(automobile_pages_urls)
fashion_reviews_df= review_rating_extract(fashion_pages_urls)
cafes_reviews_df= review_rating_extract(cafes_pages_urls)


In [ ]:
# checking the output of the extracted reviews and rating for automobile category
automobile_reviews_df.to_csv('data/raw/automobile_reviews.csv', index = False)
automobile_reviews_df.head()

In [ ]:
# checking the output of the extracted reviews and rating for fashion category
fashion_reviews_df.to_csv('data/raw/fashion_reviews.csv', index = False)
fashion_reviews_df.head()

In [ ]:
# checking the output of the extracted reviews and rating for cafe category
cafes_reviews_df.to_csv('data/raw/cafes_reviews.csv', index = False)
cafes_reviews_df.head()

#### Read Raw data and preprocess it 

Now we have sucessfully scraped all the reviews for the three bussiness categories. The next step involves cleaning and preprocessing it. we will be applying the standard text precrossing techniques such as tokenization, removal of Non-ASCII characters, remove stopwords and lemmaization. 

Tokenization breaks the sentences into individual tokens while removing the stopwords and Non-ASCII characters will ensure that the we only have the words that convey some meaning about the nature of the review. The final step is Lemmatization which essentially converts all words to their roots or lemma. For example the words plays, played, playing all are the modified form of the root word 'play' and hence can be reduced to it by the process of lemmatization.

In [ ]:
automobile_reviews_df = pd.read_csv('data/raw/automobile_reviews.csv')
fashion_reviews_df = pd.read_csv('data/raw/fashion_reviews.csv')
cafes_reviews_df = pd.read_csv('data/raw/cafes_reviews.csv')

In [ ]:
# Functions to preprocess the data 

# Function to remove non-ascii characters from the text
def _removeNonAscii(s): 
    return "".join(i for i in s if ord(i)<128)

# function to remove the punctuations, apostrophe, special characters using regular expressions

def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = text.replace('(ap)', '')
    text = re.sub(r"\'s", " is ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    text = re.sub('[^a-zA-Z ?!]+', '', text)
    text = _removeNonAscii(text)
    text = text.strip()
    return text

# stop words are the words that convery little to no information about the actual content like the words:the, of, for etc

def remove_stopwords(word_tokens):
    filtered_sentence = [] 
    stop_words = stopwords.words('english')
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    return filtered_sentence

# splitting a string, text into a list of tokens
tokenizer = RegexpTokenizer(r'\w+')
def tokenize(x): 
    return tokenizer.tokenize(x)

# lemmatization of the reviews

# function to convert nltk tag to wordnet tag
lemmatizer = WordNetLemmatizer()
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None
    
def detokenize(tokens):
    sent = TreebankWordDetokenizer().detokenize(tokens)
    return sent 

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

#### Applying all the preprocessing Functions to all the categories and storing the clean & preprocessed File

In [ ]:
# applying all of these functions to the our dataframe 
automobile_reviews_df['precrocessed_review'] = automobile_reviews_df['Review'].map(clean_text)
automobile_reviews_df['precrocessed_review'] = automobile_reviews_df['precrocessed_review'].map(tokenize)
automobile_reviews_df['precrocessed_review'] = automobile_reviews_df['precrocessed_review'].map(remove_stopwords)
automobile_reviews_df['precrocessed_review'] = automobile_reviews_df['precrocessed_review'].map(detokenize)
automobile_reviews_df['precrocessed_review'] = automobile_reviews_df['precrocessed_review'].map(lemmatize_sentence)

# checking the clean data 
automobile_reviews_df.head()

In [ ]:
# applying all the functions to the second category of data

fashion_reviews_df['precrocessed_review'] = fashion_reviews_df['Review'].map(clean_text)
fashion_reviews_df['precrocessed_review'] = fashion_reviews_df['precrocessed_review'].map(tokenize)
fashion_reviews_df['precrocessed_review'] = fashion_reviews_df['precrocessed_review'].map(remove_stopwords)
fashion_reviews_df['precrocessed_review'] = fashion_reviews_df['precrocessed_review'].map(detokenize)
fashion_reviews_df['precrocessed_review'] = fashion_reviews_df['precrocessed_review'].map(lemmatize_sentence)

# checking the clean data 
fashion_reviews_df.head()

In [ ]:
# Applying all the functions to the third category of data

cafes_reviews_df['precrocessed_review'] = cafes_reviews_df['Review'].map(clean_text)
cafes_reviews_df['precrocessed_review'] = cafes_reviews_df['precrocessed_review'].map(tokenize)
cafes_reviews_df['precrocessed_review'] = cafes_reviews_df['precrocessed_review'].map(remove_stopwords)
cafes_reviews_df['precrocessed_review'] = cafes_reviews_df['precrocessed_review'].map(detokenize)
cafes_reviews_df['precrocessed_review'] = cafes_reviews_df['precrocessed_review'].map(lemmatize_sentence)

# checking the clean data 
cafes_reviews_df.head()

From the above outputs from the respective categories, it is evident that we have now sucessfully extracted and cleaned

In [ ]:
# Saving the clean & Preprocessed data in a new CSV file

automobile_reviews_df.to_csv('data/clean/automobile_reviews.csv', index = False)
fashion_reviews_df.to_csv('data/clean/fashion_reviews.csv', index = False)
cafes_reviews_df.to_csv('data/clean/cafes_reviews.csv', index = False)

###  <font color='red'>Section 2:</font> Building a classification Model for each of the Three categories

##### Defining a global function for calculating and printing the evaluation measures of a model

In [ ]:
accuracy_list= []
train_time_list=[]
model_list = []
precision_list=[]
recall_list=[]
error_list=[]
fp_list=[]
specificity_list=[]
f_score_list=[]

def reset_lists():
    accuracy_list.clear()
    train_time_list.clear()
    model_list.clear()
    precision_list.clear()
    recall_list.clear()
    error_list.clear()
    fp_list.clear()
    specificity_list.clear()
    f_score_list.clear()

# Function to evaluate performance of a model
def evaluate_and_show(Y_test,Y_Pred, training_time, model):
    confusion_mat=confusion_matrix(Y_test,Y_Pred)
    model_list.append(model)
    print("The Training time for the model: %0.3fs" % training_time)
    train_time_list.append(" %0.3fs" % training_time)
    """This method shows different rates calculated using confusion matrix"""
    total = sum(sum(confusion_mat))

    #TN = True NO - accurate negative review 
    #TY = True Yes - accurate positive review
    #FN = False No - inacurate negative review : we have to evaluate how good our classifyer is 
    #FY = False Yes - inacurate positive review : we have to evaluate how good our classifyer is 
    TN, FN = confusion_mat[0][0], confusion_mat[1][0]
    TY, FY = confusion_mat[1][1], confusion_mat[0][1]
    
    # Evaluation and CALCULATIONS of parameters using confusion matrix
    # Accuracy = Overall, how often is the classifier correct
    accuracy = (TN + TY)/ total # true possitive + true negative 
    accuracy_list.append(accuracy*100) 
    print("Accuracy = {:.2f} %".format(accuracy*100))
    
    #Error % = Overall, how often is it wrong
    error_rate = 1 - accuracy
    error_list.append(error_rate)
    print("Error % = {:.2f} %".format(error_rate*100))
    
    #Precision: When it predicts yes, how often is it correct? 
    precision = TY/(FY + TY)
    precision_list.append(precision*100)
    print("Precision = {:.2f} %".format(precision*100))  

    #Recall (Sensitivity): When it's actually yes, how often does it predict yes 
    Recall = TY/(TY + FN)
    recall_list.append(Recall*100)
    print("Recall = {:.2f} %".format(Recall*100))
    
    # F1 Score : harmonic mean of Precsison and recall
    f_score_list.append(f1_score(Y_test,Y_Pred, pos_label='positive'))
    print("F1-Score = {:.2f}".format(f1_score(Y_test,Y_Pred, pos_label='positive')))
    
    #False Yes Rate: When it's actually no, how often does it predict yes
    FY_rate = FY/(TN + FY)
    fp_list.append(FY_rate*100)
    print("False Positive rate = {:.2f} %".format(FY_rate*100))
    
    #True No Rate(Specificity): When it's actually no, how often does it predict no
    Specificity = TN/(TN + FY)
    specificity_list.append(Specificity*100)
    print("Specificity/True Negative rate = {:.2f} %".format(Specificity*100))
    
    # Print the plot for Confusion matrix 
    fig, ax = plt.subplots(figsize=(7,7))  # figsize is the size of the figure
    group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_counts = ["{0:0.0f}".format(value) for value in
                    confusion_mat.flatten()]
    df_cm = pd.DataFrame(confusion_mat, ['NEG', 'POS'], ['NEG', 'POS'])
    labels = [f"{v1}\n{v2}" for v1, v2 in
              zip(group_names,group_counts)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(df_cm, annot=labels, fmt='')

    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title("Confusion Matrix",fontsize=15)
    plt.rcParams.update({'font.size': 13})
    plt.show()
  
 


Approach: In this section we have three categories of review and we will be building a classification model for each of category. Although the assignment requires us to use any one algorithm, I have built the model using using all the three algorithms namely, Naive Bayes, Logistic Regression and Random Forest in order to see how these models perform on each of the category and then see the best performing model.

### Category A: Automobile Review

In [ ]:
reset_lists()
# reading the clean data
automobile_reviews_df = pd.read_csv('data/clean/automobile_reviews.csv')

In [ ]:
# checking the distribuion of the class labels
print(automobile_reviews_df['Rating'].value_counts())

We see that the distribution of the classes in this category is slighlty imbalanced as there are more positive class labels in the dataset. Althought the skewness is small we will still use F1-Score along with the model's accuracy as a measure for evaluating the performnace in this case as accuracy alone can yeild a misleading result. 

#### Model 1: Random Forest Classifier  

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

In [ ]:
# calculating the TFIDF for the text review

tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(automobile_reviews_df['precrocessed_review']).toarray()

# Splitting it into train-test Split (70% for training and remaining 30% for testing)

y= automobile_reviews_df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=66)

Throughout the notebook for all the models, I have used TFIDF vectorizer in order to convert review text into the corresponding numerical form. There are several approaches of doing this task namely, bag of words (term frequency method), TFIDF ( Term Frequency Inverse document Frequency), Word embedding etc. I have chosen TFIDF because it is effectiveness and simiplicity. The bag of words approach yeilds good results when we converting text to numbers. However, it suffers from one major drackback. The Bag of words approach assigns a score to the words based on their occurrence in a particular document. It ignores the fact that the same word might be having a high frequency of occurrence in other documents as well therby conveying less information. TFIDF on the other hand is better at this because it multiplys the term frequency of a word by the inverse document frequency. The TF stands for "Term Frequency" while IDF stands for "Inverse Document Frequency". Therfore, the TFIDF score for a specific word in a particular document is higher if and only if the frequency of occurrence of that word is higher in that specific document but lower in all the other documents.

Furthermore, I have restricted the maximum number of features to 1500 only because of two reasons:
1. A review can contain hundreds of words and not all of the are significant. We dont want to calculate the TFIDF score for each and every word
2. We have to build a model that can be tested on a completely differnt dataset in section 3 therfore, if we do not specify the limit at first place while building the model we will get an error because the number of features in the trained model will not match the number of features in the test dataset.


Finally, In order to get the accurate estimate for the model's performance, we need to adopt simple hold out strategy where in we divide our data into training and testing sets. We train our model on the train set adn will test the trained model on the test set. To do so, we will use the train_test_split utility from the sklearn.model_selection library. In this notebook, all models have been trained on 70% of the dataset termed as Trainig set adn 30% of test data.

Note: I have used pickle to save the trained model for future use as it becomes very convinient to just load the model whenever needed.

In [ ]:
# Random Forrests
print("Training Random forest Classifier.......")
classifier = RandomForestClassifier(n_estimators=3000, random_state=42)
t = time()
classifier.fit(X_train, y_train)
training_time = time() - t
print("Training Complete !")
y_pred = classifier.predict(X_test)
#saving the model for use in task 3 
pickle.dump(classifier, open('data/models/rfc_automobile.sav', 'wb'))
# Genrating the COnfusion Matrix 
conf_mat=confusion_matrix(y_test,y_pred)
# for random forrests
evaluate_and_show(y_test, y_pred, training_time, 'Random Forests')

We can see that the Random Forest classification performs quite well in the classification task with the overall accuracy of 86.33% and F1-Score of 0.89. The F1-score is the harmonic mean of precison and recall. Another reason we focus on F1-Score is because here both the positive and the negative classes are significant and therefore, we want to find an optimal blend of precision and recall we can combine the two metrics using what is called the F1 score.
The train train of the model even for a smaller dataset is quite high for Random forest, this is because the random forest uses ensemble learning and henceforth a large number of decison trees are used for the classification.

#### Model 2 - Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
#Naivebayes
naive_bayes_classifier = MultinomialNB()
t = time()
naive_bayes_classifier.fit(X_train, y_train)
pickle.dump(naive_bayes_classifier, open('data/models/nbc_automobile.sav', 'wb'))
training_time = time() - t
y_pred = naive_bayes_classifier.predict(X_test)

In [ ]:
evaluate_and_show(y_test, y_pred, training_time, 'Naive Bayes')

In the above results, we can see that Naive bayes performed very well in the classification task. The accuracy of the model is 91.67 along with F1-Score of 0.93 which is very good. Moreover, given the low training time of the model, the overall performance of this model is one best that we can achieve.
The High count of True positive and True negatives in the confusion matrix above further confirm the fact.

#### Model 3 - Logistic Regression Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
# define model
modelLR = LogisticRegression(solver='lbfgs')
# fit model
t = time()
modelLR.fit(X_train, y_train)
training_time = time() - t
# save the model for future use in section 3
pickle.dump(modelLR, open('data/models/lr_automobile.sav', 'wb'))
# make predictions
y_pred = modelLR.predict(X_test)
# evaluate predictions
evaluate_and_show(y_test, y_pred, training_time, 'Logistic Regression')

The performance of Logistic regression is almost comparable to that of Naive Bayes. The model achieves a very high accuracy and F1-score of 90.33 and 0.92 respectively

In [ ]:
def model_compare():
    performance_df = pd.DataFrame()
    performance_df['Model']=model_list
    performance_df['Accuracy']=accuracy_list
    performance_df['Precision']=precision_list
    performance_df['Recall']=recall_list
    performance_df['F1-Score']=f_score_list
    performance_df['Specificity']=specificity_list
    performance_df['False Positive Rate']=fp_list
    performance_df['Train Time']= train_time_list

    th_props = [
        ("font-size", "120%"),
        ('text-align', 'center'),
        ('font-weight', 'bold'),
        ('color', '#6d6d6d'),
        ('background-color', '#f7f7f9')
        ]

        # Set CSS properties for td elements in dataframe
    td_props = [
        ('font-size', '11px')
         ]

        # Set table styles
    styles = [
        dict(selector="th", props=th_props),
        dict(selector="td", props=td_props)
        ]
    
    if(len(set(train_time_list))==1):
        return (performance_df.style
                .highlight_min(subset=['False Positive Rate'], color='lightgreen')
                .set_caption('SUMMARY OF THE PERFORMANCE OF ALL THE TESTED MODELS IN SECTION 3')
                .set_table_styles(styles)).hide_index()
    
    else: 
        return(performance_df.style
            .highlight_max(subset=['Accuracy','F1-Score'], color='lightgreen')
            .highlight_min(subset=['Accuracy','F1-Score'], color='pink')
            .set_caption('The Best Performing Model in Green and Worst Performing in Red')
            .set_table_styles(styles)).hide_index()
    
   

In [ ]:
model_compare()

The above table provides a comparasion between all the three alogithms when applied on the automobile category. It is evedent from the table for automobile category the best performing model is naive bayes while the worst performing model is Random Forests. Since we are primarily focussing on just the two measures namely, accuracy and F1-score I have highlighted only those two fields. 
The end results ar not quite what I was expecting. I expected Random forests to perform much better since it employs the ensemble learning technnique which genrally speak yeilds better results that the individual classification algorithms but that was not the case over here. In fact, Random forest Classifier performed the worst of the three. It would be interesting to see how it fairs in the other caategories in the subsequent sections.

### Category B: Fashion Review

In [ ]:
reset_lists()
fashion_reviews_df = pd.read_csv('data/clean/fashion_reviews.csv')

In [ ]:
# checking the distribuion of the class labels
print(fashion_reviews_df['Rating'].value_counts())

Just like the previous category; Automobile, the class labels in this category too are sckewed in the favor of positive class and therfore, we will continue with our strategy of using F1-score along with the accuracy score to evaluate the performance of the model. At prima-facie we see that the businesses in the automoobile and fashion category are doing quite well as both seem to have more positive reviews than the negative ones.

#### Model 1 - Random Forests Classifier

In [ ]:
# calculating the TFIDF for the text review

tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(fashion_reviews_df['precrocessed_review']).toarray()

# Splitting it into train-test Split (70% for training and remaining 30% for testing)

y= fashion_reviews_df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=66)

In [ ]:
# Random Forrests
print("Training Random forest Classifier.......")
classifier = RandomForestClassifier(n_estimators=3000, random_state=42)
t = time()
classifier.fit(X_train, y_train)
training_time = time() - t
print("Training Complete !")
y_pred = classifier.predict(X_test)
#saving the model for use in task 3 
pickle.dump(classifier, open('data/models/rfc_fashion.sav', 'wb'))
# for random forrests printing the resuts and performance
evaluate_and_show(y_test, y_pred, training_time, 'Random Forests')

We see that the perfromace of Random Forest reamins consistent with the overall accuracy of 85.67% and the F1-score of 0.89 on the 30% of the test data from the fashion category. The taining time hoverever has increased in this category as compared to the previous  category.   

#### Model 2 - Naive Bayes Classifier

In [ ]:
#Naivebayes
naive_bayes_classifier = MultinomialNB()
t = time()
naive_bayes_classifier.fit(X_train, y_train)
pickle.dump(naive_bayes_classifier, open('data/models/nbc_fashion.sav', 'wb'))
training_time = time() - t
y_pred = naive_bayes_classifier.predict(X_test)
evaluate_and_show(y_test, y_pred, training_time, 'Naive Bayes')

The performnace of Naive Bayes for Fashion category is not as good as that for the Automobile category. Naive bayes performs just as well as the Random Forest over here.

#### Model 3 - Logistic Regression Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
# define model
modelLR = LogisticRegression(solver='lbfgs')
# fit model
t = time()
modelLR.fit(X_train, y_train)
training_time = time() - t
# save the model for a use later on 
pickle.dump(modelLR, open('data/models/lr_fashion.sav', 'wb'))
# make predictions
y_pred = modelLR.predict(X_test)
# evaluate predictions
evaluate_and_show(y_test, y_pred, training_time, 'Logistic Regression')

In [ ]:
model_compare()

For the Automobile category, we can see that all the three models yeild more or less the same results both in terms of the accuracy as well as the F1 score. There is nothing much to differentiate their performance, if we had to pick one model, it would be Logistic Regression. It appears that the Fashion category contains some tricky reviews that has lead to a drop in performance of all of the three models compared to the Automobile category.


### Category C: Cafe Review

In [ ]:
reset_lists()
cafe_reviews_df = pd.read_csv('data/clean/cafes_reviews.csv')

In [ ]:
# checking the distribuion of the class labels
print(cafe_reviews_df['Rating'].value_counts())

The Data in this category is skewed to a large degree towards the postive class. The ratio seems to be perticularly high for the cafe category where there are 1462 positive reviews as opposed to just 538 negative reviews. In such cases there is a risk that the classifier only learns to classify the positive reviews and labels everything as positive. Therefore, we will be focssuing on F1 score here and along with it we will also calculate the Area Under the Curve (AUC) of the  Reciever- operator characteristic with cross validation. 

#### Model 1 - Random Forest Classifier

In [ ]:
# calculating the TFIDF for the text review

tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(cafe_reviews_df['precrocessed_review']).toarray()

# Splitting it into train-test Split (70% for training and remaining 30% for testing)

y= cafe_reviews_df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=66)

In [ ]:
# Random Forrests
print("Training Random forest Classifier.......")
classifier = RandomForestClassifier(n_estimators=3000, random_state=42)
t = time()
classifier.fit(X_train, y_train)
training_time = time() - t
print("Training Complete!")
y_pred = classifier.predict(X_test)
#saving the model for use in task 3 
pickle.dump(classifier, open('data/models/rfc_cafe.sav', 'wb'))

# for random forrests printing the resuts and performance
evaluate_and_show(y_test, y_pred, training_time, 'Random Forests')

In [ ]:
def plot_roc_curve(modelName, name):
    # roc curve and auc
    from sklearn.datasets import make_classification
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_curve
    from sklearn.metrics import roc_auc_score
    from matplotlib import pyplot
    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(y_test))]

    # predict probabilities
    lr_probs = modelName.predict_proba(X_test)
    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(y_test, ns_probs)
    lr_auc = roc_auc_score(y_test, lr_probs)
    # summarize scores
    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print( name, ': ROC AUC=%.3f' % (lr_auc))
   # print('name : ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs, pos_label='positive')
    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs, pos_label='positive')
    # plot the roc curve for the model
    pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    pyplot.plot(lr_fpr, lr_tpr, marker='.', label= name)
    # axis labels
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    # show the legend
    pyplot.legend()
    # show the plot
    pyplot.show()

In [ ]:
plot_roc_curve(classifier, 'Random Forest')

We can see from the results that the model performs very well. The accuracy and the F1-sccore of 84% and 0.90 respectively suggests that the model is accurate enough to predict the class labels. The distribution in the Confusion matrix suggests that the model has learned the features of both the classes the final evidence that the model is performing very well even on seemingly skewed dataset is from the ROC-AUC curve shown above. An excellent model which is hypothetical and does not exist, has AUC near to the 1 which means it has good measure of separability. A model that did not learn anything and is randomly assigning class labels would have the score of 0.5 and is shown by the blue line in the graph above. It means model has no class separation capacity whatsoever. In our case, the model has a score of 0.884 and at no does the model overlap with the no-skill model therefore we can confirm that the model is performing very well.

#### Model 2 - Naive Bayes Classifier

In [ ]:
#Naivebayes
naive_bayes_classifier = MultinomialNB()
t = time()
naive_bayes_classifier.fit(X_train, y_train)
pickle.dump(naive_bayes_classifier, open('data/models/nbc_cafe.sav', 'wb'))
training_time = time() - t
y_pred = naive_bayes_classifier.predict(X_test)
evaluate_and_show(y_test, y_pred, training_time, 'Naive Bayes')

In [ ]:
plot_roc_curve(naive_bayes_classifier, 'Naive Bayes')

We see that Naive bayes again does a good job here, the Accuracy and F1-score both are very good. The large number of false positives in the Confusion matix shows the underlying skewness in the data set. However the ROC-AUC curve shows that the classifier is able to distinguish between the class labels as in case with the Random Forest therfore, overall the performance is satisfactory.

#### Model 3 - Logistic Regression Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
# define model
modelLR = LogisticRegression(solver='lbfgs')
# fit model
t = time()
modelLR.fit(X_train, y_train)
training_time = time() - t
# save the model for a use later on 
pickle.dump(modelLR, open('data/models/lr_cafe.sav', 'wb'))
# make predictions
y_pred = modelLR.predict(X_test)
# evaluate predictions
evaluate_and_show(y_test, y_pred, training_time, 'Logistic Regression')

In [ ]:
plot_roc_curve(modelLR, 'Logistic Regression')

Logistic Regression performs exceedingly well, the accuracy and the F1-scores are both very good and most importantly the model has least false positives amongst all the three models. Furthermore, the model has an high ROC AUC score of 0.913 that clearly indicates its ability to distinguish between the two class labels. 

In [ ]:
model_compare()

In the final category too we see that Logistic Regression outperformed the Random Forest and barring the first category, where it was defeated by Naive bayes but only by a small margin, Logistic regresion has been the best overall classifier for classification of the reviews.
A point worth nooticing over her is that in each category, logistic regression outperformed random Forest which wasn't something I was expecting. Random Forest, employs ensemble learning as they use multiple decision trees( ensemble of decision trees) and generally speaking ensembles tend to perform better than individual classifiers. However, in this case where we were dealing with binary classification and given that the dataset was relatively small justifies the results to some extent. Logistic regression does a better job of classification in such scenarios and takes only a fraction of the total training time that a random forest would take. Now, once we have the best performing model from each category, I wanted to test this best performing model trained on one category to predict the class labels of other categories. Which brings us to the next and final section of the notebook. 

### <font color='red'>Section 3:</font> Cross Category model testing: Train on one Category and test on other two

#### 3.a: Train on Automobile and test on Fashion and Cafe

The best performing model in the Automobile category was Naive bayes. Therfore we will be begining the third section with Loading our saved model which was already trained earlier on the Automobile category. 

In [ ]:
reset_lists()

# storing the TFIDF values of all of the categories

#1.Automobile category dataset
tfidfconverter = TfidfVectorizer(max_features=1500, ngram_range=(1, 2), min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_automobile = tfidfconverter.fit_transform(automobile_reviews_df['precrocessed_review']).toarray()
Y_automobile = automobile_reviews_df['Rating']

#2.Fashion category dataset
tfidfconverter = TfidfVectorizer(max_features=1500, ngram_range=(1, 2), min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_fashion = tfidfconverter.fit_transform(fashion_reviews_df['precrocessed_review']).toarray()
Y_fashion= fashion_reviews_df['Rating']

#3.Cafe category dataset
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_cafe = tfidfconverter.fit_transform(cafe_reviews_df['precrocessed_review']).toarray()
Y_cafe= cafe_reviews_df['Rating']


##### <font color='red'>Case 1:</font> 
###### Train: Automobile Dataset
###### Test: Fashion Dataset

In [ ]:
#loading the model for Naive bayes
load_automoible_trained_nbc = pickle.load(open('data/models/nbc_automobile.sav', 'rb'))

# after loading the model we use that for prediction
y_pred = load_automoible_trained_nbc.predict(X_fashion)
evaluate_and_show(Y_fashion, y_pred, 0, 'Automibile trained NB tested on Fashion')

The Naive Bayes classifier trained on Automobile category performs very poorly on the fashion category dataset. The model has an error rate of 48.40% which is very high. Furthermore, we can also see that the model is falsely predicting a lot of positive reviews as negative and this can be seen on the Confusion matrix with 657 false negatives.
We can clearly see that while Niave Bayes performed exceedingly well when trained on a subset of Automobile Categeory and tested on a different subset within the same category. The model was not able to pick up import feaatures from that category which could be genralized and hence performed poorly when tested on a different category.

Since the performance of the best model is poor I decided to test the other two models from the same category to see how they fair. 

In [ ]:
#loading the model 
load_automoible_trained_rfc = pickle.load(open('data/models/rfc_automobile.sav', 'rb'))

In [ ]:
# Random Forrests
y_pred = load_automoible_trained_rfc.predict(X_fashion)

# for random forrests printing the resuts and performance
evaluate_and_show(Y_fashion,y_pred, 0, 'Automibile trained RF tested on Fashion ')

With the Random Forest classifier the performence improves bu only slightly and that too at a cost. There's just a 4% increase in the overall accuracy of the model comapred to Naive Bayes. The confusion Matrix and the evaluation measures show that the model has very high false positive rate which means that the model is falsely classifying the negative reviews as positive. This is not a good sign for a model. With a false positive rate of almost 71 % percent we are missing out on the important negative reviews of the cutomers about the business.

In [ ]:
#loading the model for Logistic Regression
load_automoible_trained_lr = pickle.load(open('data/models/lr_automobile.sav', 'rb'))
y_pred = load_automoible_trained_lr.predict(X_fashion)

# evaluate predictions
evaluate_and_show(Y_fashion,y_pred, 0, 'Automibile trained LR tested on Fashion')

The performance of Logistic Regression is better compared to the other two models in terms of the accuracy and F1-score but all the three models produce very number of false positives as seen in the confusion matrix. What this means is that the model is falsely classifying the negative reviews as positive this defeats the purpose of having the reivews and classifying them. Businesses would want to classify the large set of reviews into positive and negative and then focus on the negative reviews inorder to improve itself. If the classifier has a false positive rate of 75% like in ths case then it would fail to pick up on the crucial reviews. One final way to test the model is to see how it performs in the RoC AUC test.

##### <font color='red'>Case 2:</font> 
###### Train: Automobile Dataset
###### Test: Cafe Dataset

In [ ]:
#loading the model Logistic
load_automoible_trained_lr = pickle.load(open('data/models/lr_automobile.sav', 'rb'))
y_pred = load_automoible_trained_lr.predict(X_cafe)
# evaluate predictions
evaluate_and_show(Y_cafe,y_pred, 0, 'Automibile trained LR tested on Cafe')

When the automobile trained logistic Regression classifier was tested on the cafe category, the model initially seemed to have performed very well because of the high accuracy but again the model has high accuracy but that accuracy is of no use because of the very high false positive rate. As described in the previous section, the model is not effective enough to be used because of false positive rate. One reason could be that all of the three categories have nothing in common by common I mean that the words used in the autmobile buisiness reviews will be completely different than those used in the review of cafe the later will involve words that deal with food and hospitality industry. While the fashion category would involve words that are related to the fashion and beauty industry. It is cear that none of the models have learned the generic meaning of the terms as a result of which all perform very well in their own category but fail when dealing with reviews from other category.

As for the next two categories, I have tested only the best performing model from that category

#### 3.b: Train on Fashion and test on Automobile and Cafe

##### <font color='red'>Case 1:</font> 
###### Train: Fashion Dataset
###### Test: Automobile

In [ ]:

#loading the model of fashion trained dataset
load_fashion_trained_lr = pickle.load(open('data/models/lr_fashion.sav', 'rb'))
y_pred = load_fashion_trained_lr.predict(X_automobile)
# evaluate predictions
evaluate_and_show(Y_automobile,y_pred, 0, 'Fashion trained LR tested on Automobile')

The model here has an accuracy of about 58% and F1-Score of 0.71 which are poor to the say the least. furthermore, the model again has very high false positive rate of 77.66% which means a lot of the negative reviews have beeb predicted as positives. There are only 176 true negatives which is too less a number for a model to be effective.

##### <font color='red'>Case 2:</font> 
###### Train: Fashion Dataset
###### Test: Cafe Dataset

In [ ]:
y_pred = load_fashion_trained_lr.predict(X_cafe)
# evaluate predictions
evaluate_and_show(Y_cafe,y_pred, 0, 'Fashion trained LR tested on Cafe')

The test on cafe dataset seems to yeld a considerably high accuracy of approxiamtley 67 % but model does very well as it categories majority of the reviews as positive and therfore yielding a high accuracy as some of them will be correct. In the process of doing so the model is misclassifying negative reviews as positive.Like all other models we tested so far. This model too is no different from others. Such a model has misleaading accuraacy and will not be effective enough in serving its purpose. 

#### 3.c: Train on Cafe and test on Automobile and Fashion

##### <font color='red'>Case 1:</font> 
###### Train: Cafe Dataset
###### Test: Automobile Dataset

In [ ]:
#loading the model
load_cafe_trained_lr = pickle.load(open('data/models/lr_cafe.sav', 'rb'))

In [ ]:
y_pred = load_cafe_trained_lr.predict(X_automobile)
# evaluate predictions
evaluate_and_show(Y_automobile,y_pred, 0, 'Cafe trained LR tested on Automobile')

A look at the Confusion matrix shows how bad the model actually performs. There are just 8 true negatives which in it self shows us that the model is not suitable to be deployed. If we were to randomly classify reviews into positive and negatives, we would perform better than this. Thr false positive rate of 98.98 % is simply not acceptable. 

##### <font color='red'>Case 2:</font> 
###### Train: Cafe Dataset
###### Test: Fashion Dataset

In [ ]:
y_pred = load_cafe_trained_lr.predict(X_fashion)
# evaluate predictions
evaluate_and_show(Y_fashion,y_pred, 0, 'Cafe trained LR tested on fashion')

This model too suffers from the same problem like its predecessor. A vey low true negative rate and a very high false positive rate indicates that the model simply labels majority of the reviews as positve as opposed to learning the features for each class and then calssifying them. The  model model therfore is again does not genralize well and hence cannot be used to claasify reviews from the other category.

In [ ]:
model_compare()

##### <font color='red'>Section 3 Conclusion: </font> 
The above table shows a summary of all the performance metric of each of the model tested in section 3. We can clearly see that all the models suffer from the same problem:
They have very high false positive rate as is the case with all the models in the above table except 'Automobile Trained Naive Bayes on fashion Category. For any Business both positive and negative reviews are important. Positive reviews help the business understand what areas they are dooing good but earning perfect positive reviews isn’t critical to the businesses its the negative reviews that can help them grow hence negative reviews carry way more significane than the positive ones. Negative reviews provide a more realistic look of the business and actually provide more trust than having all 5 Star positive reviews. The reason why negative ones are looked at in the first place is to find out why the customers  were dissatisfied with the product or the service being offered. Hence we need to have a classifier that is able to detect both the kinds of review and what we surely do not want is a classifier that labels all the reviews as postive ones. That means we do not want a high False positive rate. If we have models with high false positive rates then it means that the model will misclassify negative reviews as positives and defeat the entire purpose of having those reviews at first place. 

We see that all the above models did very well when we trained them and tested them on the reviews from the same category but in over here when we tested them on reviews from other category all of them except one produced significantly high false positive rates. The one that did not had an accuracy of 51.6% which is as good as randomly assigning class labels to the reviews. One major reason for this could be the lack of similarity in the three selected categories. All three of them; automobile, fashion and cafe are very differnet from one another. They are three distinct industries and henceforth the words used in the reviews would also be very different therefore the model trained in one category does well in the same category but performs poorly on data from the other two categories.


### Conclusion: 

In section 1 of the notebook, we performed scraping of the reviews from three different categories of the businesses and stored them for the classification task. I decided to choose the automobile, Fashion, and Cafes because these three categories did not have anything in common as mentioned in the previous section. I wanted to check how the models will perform on categories that were the most dissimilar.

In section 2, I trained and tested all three algorithms ( Random Forests, Naïve Bayes, Logistic Regression) on each of the three categories to examine the best performing model for each category. Generally speaking, for each of the categories, logistic regression outperformed random Forest. This was not something which I was expecting. Random Forest, employs ensemble learning as they use multiple decision trees( ensemble of decision trees), and generally speaking ensembles perform better than the individual classifiers . However, in this case given the relatively small size of the dataset and given the fact it involves only two classes (binary classification) the results can be justified. Logistic regression does a better job of classification in such scenarios and it is evident here. Now, once I had the best performing model from each category, I wanted to test this best performing model trained on one category to predict the class labels of other categories.

The best performing model in the case of the automobile category turned out to be Naïve Bayes though the difference in performance for Naïve Bayes and Logistic regression was not much, nevertheless Naïve Bayes yielded best results for the automobile category. Following which I tested this model which was trained on Automobile on the other two categories I got a dismal accuracy of around 45% on both categories i.e. cafe and fashion this suggests that while naive Bayes performs exceedingly well on the test train split from the same category, it does not do well while predicting the labels of other two categories. This means it does not generalize across the categories. Therefore, I decided to test the other two models from the automobile category in fashion to see how they fare. It was found that the other two models did not perform well either. An important evaluation measure in this category is the False positive rate and all of the built models performed poorly in this evaluation.

Thus, we can conclude that logistic regression is best choice of algorithm when the data is from the same category and we test it on the same category. I had selected the most dissimilar categories in order to see how well the models generalize by picking up positive and negative features from the dataset and then using the same features to predict the labels for another category but after sufficient testing, it can be concluded that the model’s performance is dependent  on the similarity on the words between the train and test data set.

In future, we could add bi-gram and tri-gram analysis and then select the most informative features from it. Along with that we could add sentiment analysis and count the number of positive and negative words in a review by comparing it to one of the available sentiment word list and then run classifer on it. I belive such a technique would allow the models to generalize better.
